<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-IllinoisGRMHD-NRPyfied: `ADM_3METRIC` and `CONF_METRIC`

## Authors: Leo Werneck & Zach Etienne

<font color='red'>**This module is currently under development**</font>

## In this tutorial notebook we implement the `ADM_3METRIC` and `CONF_METRIC` variables used by the NRPyfied version of `IllinoisGRMHD`

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

0. [Step 0](#loading_modules): **Loading necessary Python/NRPy+ modules**
1. [Step 1](#adm_3metric_vars): **The `ADM_3METRIC` variable declaration**
1. [Step 2](#conf_metric_vars): **The `CONF_METRIC` variable declaration**
1. [Step 3](#compute_gamma_inv_and_sqrtgamma): **The `compute__ADM_gammaUU_and_sqrtgamma__in_terms_of__ADM_3METRIC.h` file**
1. [Step 4](#compute_g4dd): **The `compute__g4DD__in_terms_of__ADM_3METRIC` file**
1. [Step 5](#compute_g4uu): **The `compute__g4UU__in_terms_of__ADM_3METRIC` file**
1. [Step 6](#compute__lapseinv_psi2_psi4_psi6): **The `compute__lapseinv_psi2_psi4_psi6__in_terms_of__ADM_3METRIC_and_CONF_METRIC.h` file**
1. [Step n](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='loading_modules'></a>

# Step 0: Loading necessary Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{loading_modules}$$

We now load the necessary Python and NRPy+ modules needed by this tutorial notebook.

In [1]:
# Import Python modules
import os,sys            # Python module: used for system and OS specific commands
import sympy as sp       # Python module: used for symbolic expressions
import re                # Python module: used to manipulate regular expressions

# Register NRPy+ root directory to the path
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Load NRPy+ modules
from outputC import *    # NRPy+ module: used to output sympy expressions to C
import indexedexp as ixp # NRPy+ module: used to generate indexed expressions (e.g. g_{\mu\nu})

# Set up a neat function to output the expressions to NRPy+ generated files
def NRPy_IGM_write_to_file(filename,contents):
    with open(filename,"w") as file:
        file.write("""
/* .-----------------------------------------------------------------------.
 * | This file was generated by NRPy+ for IllinoisGRMHD, as documented in: |
 * |        Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.ipynb         |
 * .-----------------------------------------------------------------------.
 * |                Author(s): Leo Werneck and Zach Etienne                |
 * .-----------------------------------------------------------------------.
 * |             Source: https://github.com/leowerneck/NRPyIGM             |
 * .-----------------------------------------------------------------------.
 *
 * File start: """+filename+""" */
\n"""+contents+"""
/* File end  : """+filename+""" */
""")

<a id='adm_3metric_vars'></a>

# Step 1: The `ADM_3METRIC` variable declaration \[Back to [top](#toc)\]
$$\label{adm_3metric_vars}$$

The `ADM_3METRIC` array contains the following quantities:

\begin{align}
{\rm ALPHA} &:= \alpha\ ,\\
{\rm BETAU} &:= \beta^{i}\ ,\\
{\rm GDD}   &:= \gamma_{ij}\ ,\\
{\rm GUPDD} &:= \gamma^{ij}\ ,\\
{\rm SQRTGAMMA} &:= \sqrt{\gamma} \ ,\ \gamma \equiv \det\left(\gamma_{ij}\right)\ .
\end{align}

We start by declaring a function that sets up these variables.

In [2]:
# Step 1: Declare basic ADM variables to be used by IllinoisGRMHD
# Step 1.a: Set spatial dimension to 3
DIM = 3

# Step 1.b: Set up alpha
alpha = sp.symbols("ADM_3METRIC[ALPHA]",real=True)

# Step 1.b: Set up beta^{i}
betaU   = ixp.zerorank2()
for i in range(DIM):
    betaU[i] = sp.symbols('ADM_3METRIC[SHIFT'+chr(ord('X')+i)+"]",real=True)

# Step 1.c: Set up gamma_{ij}
gammaDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(i,DIM):
        gammaDD[i][j] = gammaDD[j][i] = sp.symbols('ADM_3METRIC[G'+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)

# Step 1.d: Set up gamma^{ij}
gammaUU = ixp.zerorank2()
for i in range(3):
    for j in range(i,3):
        gammaUU[i][j] = gammaUU[j][i] = sp.symbols('ADM_3METRIC[GUP'+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)
        
# Step 1.e: Set up sqrt(gamma)
sqrtgamma = sp.symbols("ADM_3METRIC[SQRTGAMMA]",real=True)

<a id='conf_metric_vars'></a>

# Step 2: The `CONF_METRIC` variable declaration \[Back to [top](#toc)\]
$$\label{conf_metric_vars}$$

The `CONF_METRIC` array contains the following quantities:

\begin{align}
{\rm PHI} &:= \phi\ ,\ \text{conformal factor}\ ,\\
{\rm PSI2} &:= \psi^{2} = e^{2\phi}\ ,\ \text{useful auxiliary variable}\ ,\\
{\rm PSI4} &:= \psi^{4}\ ,\ \text{useful auxiliary variable}\ ,\\
{\rm PSI6} &:= \psi^{6}\ ,\ \text{useful auxiliary variable}\ ,\\
{\rm GAMMADD} &:= \bar\gamma_{ij}\ ,\\
{\rm GAMMAUU} &:= \bar\gamma^{ij}\ .
\end{align}

We start by declaring a function that sets up these variables.

In [3]:
# Step 2: Set the the conformal metric variables to be used by IllinoisGRMHD
# Step 2.a: Set up phi
cf_phi = sp.symbols("CONF_METRIC[PHI]",real=True)

# Step 2.b: Set up \bar\gamma_{ij}
gammabarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(i,DIM):
        gammabarDD[i][j] = gammabarDD[j][i] = sp.symbols('CONF_METRIC[GAMMA'+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)

# Step 2.c: Set up \bar\gamma^{ij}
gammabarUU = ixp.zerorank2()
for i in range(3):
    for j in range(i,3):
        gammabarUU[i][j] = gammabarUU[j][i] = sp.symbols('CONF_METRIC[GAMMAUP'+chr(ord('X')+i)+chr(ord('X')+j)+"]",real=True)

<a id='compute_gamma_inv_and_sqrtgamma'></a>

# Step 3: The `compute__ADM_gammaUU_and_sqrtgamma __in_terms_of__ADM_3METRIC.h` file \[Back to [top](#toc)\]
$$\label{compute_gamma_inv_and_sqrtgamma}$$

Now we compute the inverse ADM 3-metric $\gamma^{ij}$ from $\gamma_{ij}$, by inverting $\gamma_{ij}$ using our `symm_matrix_inverter3x3()` function from the [indexedexp.py](http://localhost:8888/edit/NRPyIGM/indexedexp.py) NRPy+ module.

In [4]:
# Step 3: The compute_ADM_gammaUU_and_sqrtgamma.h file
# Step 3.a: Compute eh inverse ADM 3-metric and the determinant of the ADM 3-metric
gammaINV, gammaDET = ixp.symm_matrix_inverter3x3(gammaDD)

# Step 3.b: Populate the ADM_3METRIC[GUPUU] and ADM_3METRIC[SQRTGAMMA]
#           with the results of our inverter
string = outputC([ gammaINV[0][0],       gammaINV[0][1],       gammaINV[0][2],
                   gammaINV[1][1],       gammaINV[1][2],       gammaINV[2][2], sp.sqrt(gammaDET)],
        ["ADM_3METRIC[GUPXX]","ADM_3METRIC[GUPXY]","ADM_3METRIC[GUPXZ]",
         "ADM_3METRIC[GUPYY]","ADM_3METRIC[GUPYZ]","ADM_3METRIC[GUPZZ]", "ADM_3METRIC[SQRTGAMMA]"],
        filename="returnstring", params="outCverbose=False")

# Step 3.c: Replace pow(blah, 2) with (blah)*(blah)
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2

# Step 3.d: Output result to file
filename = "compute__ADM_gammaUU_and_sqrtgamma__in_terms_of__ADM_3METRIC.h"
NRPy_IGM_write_to_file(filename,string)
    
print("Just generated the file: "+filename)

Just generated the file: compute__ADM_gammaUU_and_sqrtgamma__in_terms_of__ADM_3METRIC.h


<a id='compute_g4dd'></a>

# Step 4: The `compute__g4DD__in_terms_of__ADM_3METRIC.h` file \[Back to [top](#toc)\]
$$\label{compute_g4dd}$$

We now compute the ADM 4-metric, $g_{\mu\nu}$, given by (see equation 2.122 in [Baumgarte & Shapiro's Numerical Relativity](https://www.google.com/books/edition/Numerical_Relativity/dxU1OEinvRUC?hl=en&gbpv=0))

$$
g_{\mu\nu}
=
\begin{pmatrix}
-\alpha^{2} + \beta_{\ell}\beta^{\ell} & \beta_{i}\\
\beta_{j} & \gamma_{ij}
\end{pmatrix}\ .
$$

We do this with using the [BSSN/ADMBSSN_tofrom_4metric](http://localhost:8888/edit/NRPyIGM/BSSN/ADMBSSN_tofrom_4metric.py) NRPy+ module.

In [5]:
# Step 4: Compute the ADM 4-metric, g_{\mu\nu}
# Step 4.a: Load the BSSN.ADMBSSN_tofrom_4metric NRPy+ module
import BSSN.ADMBSSN_tofrom_4metric as AB4m

# Step 4.b: Compute the g_{\mu\nu} in terms of our ADM variables
AB4m.g4DD_ito_BSSN_or_ADM("ADM",gammaDD=gammaDD,betaU=betaU,alpha=alpha)

# Step 4.c: Output to file
# Step 4.c.i: Set up lists to store the expressions and output variables
exprlist = []
namelist = []
for mu in range(4):
    for nu in range(4):
        exprlist.append(AB4m.g4DD[mu][nu])
        namelist.append("g4dn["+str(mu)+"]["+str(nu)+"]")
        
# Step 4.c.ii: Convert our results to C output
string = outputC(exprlist,namelist,"returnstring", params="outCverbose=False")
# Step 4.c.iii: Replace pow(blah, 2) with (blah)*(blah)
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2

# Step 4.c.iv: Output to file
filename = "compute__g4DD__in_terms_of__ADM_3METRIC.h"
NRPy_IGM_write_to_file(filename,string)

print("Just generated the file: "+filename)

Just generated the file: compute__g4DD__in_terms_of__ADM_3METRIC.h


<a id='compute_g4uu'></a>

# Step 5: The `compute__g4UU__in_terms_of__ADM_3METRIC.h` file \[Back to [top](#toc)\]
$$\label{compute_g4uu}$$

We now compute the inverse ADM 4-metric, $g^{\mu\nu}$, given by (see equation 2.119 in [Baumgarte & Shapiro's Numerical Relativity](https://www.google.com/books/edition/Numerical_Relativity/dxU1OEinvRUC?hl=en&gbpv=0))

$$
g^{\mu\nu}
=
\begin{pmatrix}
-\alpha^{-2} & \alpha^{-2}\beta^{i}\\
\alpha^{-2}\beta^{j} & \gamma^{ij} - \alpha^{-2}\beta^i\beta^{j}
\end{pmatrix}\ .
$$

We do this with using the [BSSN/ADMBSSN_tofrom_4metric](http://localhost:8888/edit/NRPyIGM/BSSN/ADMBSSN_tofrom_4metric.py) NRPy+ module.

In [6]:
# Step 5: Compute the inverse ADM 4-metric, g^{\mu\nu}
# Step 5.a: Load the BSSN.ADMBSSN_tofrom_4metric NRPy+ module
import BSSN.ADMBSSN_tofrom_4metric as AB4m

# Step 5.b: Compute the g_{\mu\nu} in terms of our ADM variables
AB4m.g4UU_ito_BSSN_or_ADM("ADM",betaU=betaU,alpha=alpha,gammaUU=gammaUU)

# Step 5.c: Set up lists to store the expressions and output variables
exprlist = []
namelist = []
for mu in range(4):
    for nu in range(4):
        exprlist.append(AB4m.g4UU[mu][nu])
        namelist.append("g4up["+str(mu)+"]["+str(nu)+"]")
        
# Step 5.d: Convert our results to C output
string = outputC(exprlist,namelist,"returnstring", params="outCverbose=False")
# Step 5.e: Replace pow(blah, 2) with (blah)*(blah)
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2

# Step 5.f: Output to file
filename = "compute__g4UU__in_terms_of__ADM_3METRIC.h"
NRPy_IGM_write_to_file(filename,string)

print("Just generated the file: "+filename)

Just generated the file: compute__g4UU__in_terms_of__ADM_3METRIC.h


<a id='compute__lapseinv_psi2_psi4_psi6'></a>

# Step 6: The `compute__lapseinv_psi2_psi4_psi6__in_terms_of __ADM_3METRIC_and_CONF_METRIC.h` file \[Back to [top](#toc)\]
$$\label{compute__lapseinv_psi2_psi4_psi6}$$

We now compute basic auxiliary quantities, specifically $\alpha^{-1}$, $\psi^{2}$, $\psi^{4}$, and $\psi^{6}$.

In [9]:
# Step 6: Compute useful auxiliary quantities
# Step 6.a: Set up lapseinv = 1/alpha
lapseinv = 1.0/alpha

# Step 6.b: Set up psi2 = psi^{2} = exp(2*phi)
psi2 = sp.exp(2*cf_phi)

# Step 6.c: Set up psi4 = psi^{4} = psi^{2} * psi^{2}
psi4 = sp.symbols("psi2",real=True)*sp.symbols("psi2",real=True)

# Step 6.d: Set up psi6 = psi^{6} = psi^{2} * psi^{4}
psi6 = sp.symbols("psi2",real=True)*sp.symbols("psi4",real=True)

# Step 6.e: Convert our results to C output
string = outputC([ lapseinv , psi2 , psi4 , psi6],
                 ["CCTK_REAL lapseinv","CCTK_REAL psi2","CCTK_REAL psi4","CCTK_REAL psi6"],"returnstring", params="outCverbose=False")

# Step 6.f: Replace pow(blah, 2) with (blah)*(blah)
string2 = re.sub('pow\(([^,]+), 2\)', '(\\1)*(\\1)', string); string = string2

# Step 6.g: Output to file
filename = "compute__lapseinv_psi2_psi4_psi6__in_terms_of__ADM_3METRIC_and_CONF_METRIC.h"
NRPy_IGM_write_to_file(filename,string)
    
print("Just generated the file: "+filename)

Just generated the file: compute__lapseinv_psi2_psi4_psi6__in_terms_of__ADM_3METRIC_and_CONF_METRIC.h


<a id='latex_pdf_output'></a>

# Step n: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__A_i_rhs_no_gauge_terms.pdf](Tutorial-IllinoisGRMHD__A_i_rhs_no_gauge_terms.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

In [4]:
latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__NRPyfied_IGM_expressions.tex
!rm -f Tut*.out Tut*.aux Tut*.log